<a href="https://colab.research.google.com/github/seunghyunmoon2/NLP/blob/master/NLP4_TFIDF/TopicModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Edit distance

Also known as, **Levenshtein distance**, is a method to calculate the similarity between two scalar of distance of instances.

it is used  to convert a string A, which was allegedly meant to represent B, into a string B by various operations such as delete, insert, subsitute.

for example, when a string 'appie' is given, isn't it a reasonable to guess that the user meant 'apple'?

## examples

In [ ]:
# import
from nltk.metrics.distance import edit_distance

In [ ]:
# define a function that returns edit distance and perform sanity check with the library function `nltk.metrics.distance.edit_distance`

# fucntion receives two strings and returns int valued 'distance'
def my_edit_distance(str1, str2): -> int
    # create a table size of m x n
    m = len(str1)+1 # HAND
    n = len(str2)+1 # AND

    # initialize table. only the first row and column
    table = {}
    for i in range(m): table[i,0] = i
    for j in range(n): table[0,j] = j

    # fill in the matrix
    for i in range(1,m):
        for j in range(1,n):
            cost = 0 if str1[i-1] == str2[j-1] else 1
            table[i,j] = min(table[i,j-1]+1, table[i-1, j]+1, table[i-1, j-1]+ cost)
            
    return table[i,j]

In [ ]:
print('my algorithm : ',my_edit_distance('hand','and'))
print('NLTK algorithm : ',edit_distance('hand','and'))

* output
```
my algorithm : 1
NLTK algorithm : 1
```



# TF-IDF



In [ ]:
# build a model that finds the cosine similarity rank of given sentences.
import pandas as pd
import numpy as np

doc1 = 'gold truck silver'
doc2 = 'shipment of gold damaged in a fire'
doc3 = 'delivery of silver arrived in a silver truck'
doc4 = 'shipment of gold arrived in a truck'

Bow1 = doc1.split(' ')
Bow2 = doc2.split(' ')
Bow3 = doc3.split(' ')
Bow4 = doc4.split(' ')

Unique = sorted(list(set(Bow1+Bow2+Bow3+Bow4)))

fq1 = dict.fromkeys(Unique,0)
fq2 = dict.fromkeys(Unique,0)
fq3 = dict.fromkeys(Unique,0)
fq4 = dict.fromkeys(Unique,0)

for word in Bow1:
    fq1[word] += 1
for word in Bow2:
    fq2[word] += 1
for word in Bow3:
    fq3[word] += 1
for word in Bow4:
    fq4[word] += 1

print(fq1)
print(fq2)
print(fq3)
print(fq4)


table_value = np.array([list(fq1.values()),list(fq2.values()),list(fq3.values()),list(fq4.values())]).T
table = pd.DataFrame(table_value,index=Unique,columns=['fq1','fq2','fq3','fq4'])
table.loc['#words'] = [sum(fq1.values()), sum(fq2.values()), sum(fq3.values()), sum(fq4.values())]
table['DF'] = table.sum(axis=1)
print(table)


#TF
TF_value = table_value/np.array(table.loc['#words'][:4])
TF = pd.DataFrame(TF_value,index=Unique,columns=['fq1','fq2','fq3','fq4'])
print(TF)


#IDF
IDF = np.array(np.log(len(Unique)/np.array(table['DF'][:-1])))
print(IDF)

TF_IDF = np.multiply(TF_value.T, IDF).T

#Norm - innerdot - cosdistance
Norm = np.sqrt(np.sum(np.square(TF_IDF),axis=0))
# array([0.75014138, 0.65711979, 0.58844549, 0.53933091])

innerdot = np.array([np.inner(Norm[0],Norm[1]),np.inner(Norm[0],Norm[2]),np.inner(Norm[0],Norm[3])])
# array([0.49293275, 0.44141731, 0.40457443])

# cosine distance
cos_dist = np.array([(innerdot[0]/Norm[0]*Norm[1]),(innerdot[1]/Norm[0]*Norm[2]),(innerdot[2]/Norm[0]*Norm[3])])
# array([0.43180642, 0.34626809, 0.29087783])

## clearer code

In [ ]:
# TFIDF 연습
# 2020.07-21
# ----------
import nltk
import numpy as np

# 1. dictionary를 생성한다.
def makeVocab(sentences):
    words = [word for sentence in sentences for word in sentence.split()]
    words = list(set(words))
    words.sort()
    return {word: idx for idx, word in enumerate(words)}

# 2. TF를 생성한다.
def makeTF(sentences):
    vocab = makeVocab(sentences)
    tf = np.zeros((len(vocab), len(sentences)))
    for i, sentence in enumerate(sentences):
        freq = nltk.FreqDist(nltk.word_tokenize(sentence))
        for key in freq.keys():
            tf[vocab[key], i] = freq[key] / len(sentence)
    return tf

# 3. IDF를 생성한다.
def makeIDF(sentences, tf):
    df = tf.shape[1] - (tf == 0.0).sum(axis=1)
    return np.log(tf.shape[1] / (0+df))

# 4. TFIDF를 생성한다.
def makeTFIDF(sentences):
    tf = makeTF(sentences)
    idf = makeIDF(sentences, tf)
    return np.multiply(tf, idf.reshape(tf.shape[0], 1))

sentences = ['gold silver truck', 'shipment of gold damaged in a fire', 'delivery of silver arrived in a silver truck', 'shipment of gold arrived in a truck']
tfidf = makeTFIDF(sentences);
print(tfidf.round(4))

# Topic Model

> **dimension reduction** is conducted by PCA, SVD methods. SVD can be used to find *Topic Model*. 

> here we have LSA comes out. We deal with two models for Topic Model. 
1. LSA
2. LDA

First, LSA takes SVD method where as LDA will be dealt with later.

    

##  LSA

### SVD

In [ ]:
# TF-IDF matrix를 SVD로 분해한다.
# C = U.S.VT
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

statements = [
            'ruled india',
            'Chalukyas ruled Badami',
            'So many kingdoms ruled India',
            'Lalbagh is a botanical garden in India'
        ]

# TF-IDF matrix를 생성한다.
tf_vector = TfidfVectorizer(max_features = 8)
tfidf = tf_vector.fit_transform(statements)
print(tfidf.shape)

# SVD (Singular Vector Decomposition)으로 TF-IDF를 분해한다.
# U, S, VT 행렬의 의미 --> Latent Semantic Analysis (LSA)
# U 행렬 ~ 차원 = (문서 개수 X topic 개수) : 문서당 topic 분포
# S 행렬 ~ 차원 = (topic 개수 X topic 개수)
# VT 행렬. 차원 = (topic 개수 X 단어 개수) : topic 당 단어 빈도 (분포)
U, S, VT = np.linalg.svd(tfidf.toarray(), full_matrices = True)

print(U.round(2), '\n')
print(S.round (2), '\n')
print(VT.round(2), '\n')

# S를 행렬 형태로 변환한다.
s = np.zeros(tfidf.shape)
s[:S.shape[0], :S.shape[0]] = np.diag(S)
print(s.round(2), '\n')

### LSA

In [ ]:
# Latent Semantic Analysis (LSA)
# ------------------------------
import numpy as np
import re
import pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

#from sklearn.datasets import fetch_20newsgroups

# news data를 읽어와서 저장해 둔다.
#newsData = fetch_20newsgroups(shuffle=True, 
#                              random_state=1, 
#                              remove=('headers', 'footers', 'quotes'))

#with open('./dataset/news.data', 'wb') as f:
#    pickle.dump(newsData , f, pickle.HIGHEST_PROTOCOL)

# 저장된 news data를 읽어온다.
with open('./dataset/news.data', 'rb') as f:
    newsData  = pickle.load(f)

# 첫 번째 news를 조회해 본다.
news = newsData.data
print(len(news))
print(news[0])

# news 별로 분류된 target을 확인해 본다.
print(newsData.target_names)
print(len(newsData.target_names))

# preprocessing.
# 1. 영문자가 아닌 문자를 모두 제거한다.
news1 = []
for doc in news:
    news1.append(re.sub("[^a-zA-Z]", " ", doc))

# 2. 불용어를 제거하고, 모든 단어를 소문자로 변환하고, 길이가 3 이하인 
# 단어를 제거한다
stop_words = stopwords.words('english')
news2 = []
for doc in news1:
    doc1 = []
    for w in doc.split():
        w = w.lower()
        if len(w) > 3 and w not in stop_words:
            doc1.append(w)
    news2.append(' '.join(doc1))
    
print(news2[0])

# TF-IDF matrix를 생성한다.
tf_vector = TfidfVectorizer(max_features = 500)
tfidf = tf_vector.fit_transform(news2)
print(tfidf.shape)

vocab = tf_vector.get_feature_names()
print(vocab[:20])

# Latent Semantic Analysis (LSA)
# ------------------------------
svd = TruncatedSVD(n_components = len(newsData.target_names), n_iter=1000)
svd.fit(tfidf)

U = svd.fit_transform(tfidf) / svd.singular_values_
VT = svd.components_
S = np.diag(svd.singular_values_)

# 문서 별 Topic 번호를 확인한다. (문서 10개만 확인)
for i in range(10):
    print('문서-{:d} : topic = {:d}'.format(i, np.argmax(U[i:(i+1), :][0])))
    
# VT 행렬에서 topic 별로 중요 단어를 표시한다
for i in range(len(VT)):
    idx = np.flipud(VT[i].argsort())[:10]
    print('토픽-{:2d} : '.format(i+1), end='')
    for n in idx:
        print('{:s} '.format(vocab[n]), end='')
    print()

newsData.keys()

# 문서별로 분류된 코드를 확인해 본다.
# x, y : 문서 번호
def checkTopic(x, y):
    print("문서 %d의 topic = %s" % (x, newsData.target_names[newsData.target[x]]))
    print("문서 %d의 topic = %s" % (y, newsData.target_names[newsData.target[y]]))

checkTopic(1, 6)
checkTopic(0, 2)

## LDA

> LDA는 결합확률분포로 취급하고(문서, 단어 조합의 관점에서) **Gibbs' sampling**의 샘플링 기법으로 근사치를 구한다.

> 두개의 다항분포를 추정하고, 해당 문서를 특정 topic에 할당하는 알고리즘.

> 문서는 토픽 집합에 속한 단어들로 구성되고, 한 문서에 등장하는 단어들은 여러 토픽들과 연계되어 있다. -> 토픽 분포


### algorithm explanation

> How LDA algorithm workds : from a random document of its topic we want to know, we observe it's word distribution( `w` ). and we can back track (`theta` and `beta`) with `w`. 

> Where `beta` represents the likelihood of the topic given distribution of words and `theta` represents the likelihood of the topic of the document.

In [ ]:
# Latent Dirichlet Allocation (LDA)
# ---------------------------------
import numpy as np
import re
import pickle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

#from sklearn.datasets import fetch_20newsgroups

# news data를 읽어와서 저장해 둔다.
#newsData = fetch_20newsgroups(shuffle=True, 
#                              random_state=1, 
#                              remove=('headers', 'footers', 'quotes'))

#with open('./dataset/news.data', 'wb') as f:
#    pickle.dump(newsData , f, pickle.HIGHEST_PROTOCOL)

# 저장된 news data를 읽어온다.
with open('./dataset/news.data', 'rb') as f:
    newsData  = pickle.load(f)

# 첫 번째 news를 조회해 본다.
news = newsData.data
print(len(news))
print(news[0])

# news 별로 분류된 target을 확인해 본다.
print(newsData.target_names)
print(len(newsData.target_names))

# preprocessing.
# 1. 영문자가 아닌 문자를 모두 제거한다.
news1 = []
for doc in news:
    news1.append(re.sub("[^a-zA-Z]", " ", doc))

# 2. 불용어를 제거하고, 모든 단어를 소문자로 변환하고, 길이가 3 이하인 
# 단어를 제거한다
stop_words = stopwords.words('english')
news2 = []
for doc in news1:
    doc1 = []
    for w in doc.split():
        w = w.lower()
        if len(w) > 3 and w not in stop_words:
            doc1.append(w)
    news2.append(' '.join(doc1))
    
print(news2[0])

# TF-IDF matrix를 생성한다.
tf_vector = TfidfVectorizer(max_features = 500)
tfidf = tf_vector.fit_transform(news2)
print(tfidf.shape)

vocab = tf_vector.get_feature_names()
print(vocab[:20])

# Latent Dirichlet Allocation (LDA)
# ---------------------------------
# Return 값이 Document-Topic distribution이다.
# iteration 횟수가 max_iter까지 가면 아직 수렴하지 않은 것이다.
# 아직 수렴하지 않은 경우 mat_iter를 증가시켜야 한다.
# mat_iter를 증가시켜도 수렴하지 못하는 경우는 preprocessing 등을 좀 더 정밀하게 해야 한다.
# perplexity는 감소해야 한다.
# perplexity의 역할 : elbow 같은 역할
# n_components는 적절히 바꿔줄 수도 있음. perplexity 변화에 따라서
# evaluate_every - 5번 반복마다 성능 평가 & 결과 출력
model = LDA(n_components = len(newsData.target_names), 
            learning_method='online', 
            evaluate_every=5, 
            max_iter=1000, 
            verbose=1)

doc_topic = model.fit_transform(tfidf)

# 문서 별 Topic 번호를 확인한다. (문서 10개만 확인)  ## theta
for i in range(10):
    print('문서-{:d} : topic = {:d}'.format(i, np.argmax(doc_topic[i:(i+1), :][0])))

# topic_term 행렬에서 topic 별로 중요 단어를 표시한다  ## beta
topic_term = model.components_
for i in range(len(topic_term)):
    idx = np.flipud(topic_term[i].argsort())[:10]
    print('토픽-{:2d} : '.format(i+1), end='')
    for n in idx:
        print('{:s} '.format(vocab[n]), end='')
    print()

newsData.keys()

# 문서별로 분류된 코드를 확인해 본다.
# x, y : 문서 번호
def checkTopic(x, y):
    print("문서 %d의 topic = %s" % (x, newsData.target_names[newsData.target[x]]))
    print("문서 %d의 topic = %s" % (y, newsData.target_names[newsData.target[y]]))

checkTopic(1, 5)
checkTopic(0, 2)
checkTopic(4, 7)